In [2]:
import numpy as np

In [3]:
def is4ConnectedBoundaryPixel(labels, i, j):
    if i > 0 and labels[i][j] != labels[i-1][j]:
        return True
    if i < labels.shape[0] - 1 and labels[i][j] != labels[i+1][j]:
        return True
    if j > 0 and labels[i][j] != labels[i][j-1]:
        return True
    if j < labels.shape[1] - 1 and labels[i][j] != labels[i][j+1]:
        return True
    return False

In [ ]:
def computeIntersectionMatrix(labels, gt):
    superpixels = max(labels);
    gt_segments = max(gt);
    for i in range(labels.shape[0]):
        for j in range(labels.shape[1]):
            if labels[i][j] > superpixels:
                superpixels = labels[i][j]
            if gt[i][j] > gt_segments:
                gt_segments = gt[i][j]
    superpixels+=1;
    gt_segments+=1;
#     superpixel_sizes.resize(superpixels, 0);
#     gt_sizes.resize(gt_segments, 0);
    
    intersection_matrix = np.zeros([gt_segments, superpixels])
#     // Rember to reset as not done in create.
    superpixel_sizes = np.zeros([superpixels])
    gt_sizes = np.zeros([gt_sizes])
    for i in range(gt.shape[0]):
        for j in range(gt.shape[1]):
            intersection_matrix[gt[i][j]][labels[i][j]] += 1
            superpixel_sizes[i][j] += 1
            gt_sizes[i][j] += 1

    return intersection_matrix, superpixel_sizes, gt_sizes

In [4]:
def boundaryRecall(gt, pred, d):
    assert gt.shape[0] == pred.shape[0] and gt.shape[1] == pred.shape[1]
    h = gt.shape[0]
    w = gt.shape[1]
    r = np.round(d*(h*h+w*w)**(1/2))
    tp = 0
    fn = 0
    for i in range(h):
        for j in range(w):
            if is4ConnectedBoundaryPixel(gt, i, j):
                pos = False
                for k in range(max(0, i-r), min(h - 1, i + r) + 1):
                    for l in range(max(0, j - r), min(w - 1, j + r) + 1):
                        if is4ConnectedBoundaryPixel(pred, k, l):
                            pos = True
                if pos:
                    tp += 1
                else:
                    fn += 1
    if tp + fn > 0:
        return tp/(tp + fn)
    
    return 0

In [ ]:
def computeAchievableSegmentationAccuracy(gt, pred):
    assert gt.shape[0] == pred.shape[0] and gt.shape[1] == pred.shape[1]
    h = gt.shape[0]
    w = gt.shape[1]
    n = h*w
    intersection_matrix, superpixel_sizes, gt_sizes = computeIntersectionMatrix(pred, gt)
    acc = 0
    for j in range(intersection_matrix.shape[1]):
        mx = 0
        for i in range(intersection_matrix.shape[0]):
            if intersection_matrix[i][j] > mx:
                mx = intersection_matrix[i][j]
        accuracy += mx
    return accuracy/N

In [ ]:
def computeUndersegmentationError(gt, pred):
    assert gt.shape[0] == pred.shape[0] and gt.shape[1] == pred.shape[1]
    h = gt.shape[0]
    w = gt.shape[1]
    n = h*w
    intersection_matrix, superpixel_sizes, gt_sizes = computeIntersectionMatrix(pred, gt)
    error = 0
    for i in range(intersection_matrix.shape[1]):
        mn = float('inf')
        for i in range(intersection_matrix.shape[0]):
            tmp = superpixel_sizes[j] - intersection_matrix[i][j]
            if tmp  < mn:
                mn = tmp
                